In [1]:
!pip install opendatasets

In [3]:
import opendatasets as od
od.download("https://www.kaggle.com/datasets/ninadaithal/imagesoasis/download?datasetVersionNumber=1" , force = True )


100%|██████████| 1.23G/1.23G [00:18<00:00, 72.6MB/s]


In [ ]:
directory = '/content/imagesoasis/Data'


In [ ]:
from torch.nn.parallel import DataParallel

In [ ]:
import torch
import matplotlib.pyplot as plt
from PIL import Image
from torchvision.datasets import ImageFolder
from torchvision.transforms import ToTensor, Resize, Normalize, RandomHorizontalFlip, RandomCrop

import numpy as np
from torch.utils.data import random_split
from torch.utils.data import DataLoader, WeightedRandomSampler

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader


print("Imported the necessary libraries")

In [ ]:
dataset = ImageFolder(directory, transform=ToTensor())

print('Dataset created and transformed to the torch dataset structure')

Dataset created and transformed to the torch dataset structure


In [ ]:
class_counts = torch.bincount(torch.tensor(dataset.targets))
class_count = np.array(class_counts)
print('Class Count : ', class_count)

Class Count :  [ 5002   488 67222 13725]


In [ ]:
val_size = int(len(dataset) * 0.15)
train_size = len(dataset) - val_size

train_ds, val_ds = random_split(dataset, [train_size, val_size])
print(f'Length of Training Dataset : {len(train_ds)}, Length of Validation Dataset : {len(val_ds)}')

Length of Training Dataset : 73472, Length of Validation Dataset : 12965


In [ ]:
class_weights_ = 1.0 / class_count
class_weights = class_weights_ / sum(class_weights_)
weight_array = torch.zeros(train_size)
i = 0
for _,label in train_ds:
    weight_array[i] = class_weights[label]
    i += 1
print('Calculated the class weights')

Calculated the class weights


In [9]:
total_samples = 50000
weighted_sampler = WeightedRandomSampler(weights = weight_array, replacement = True, num_samples = total_samples)
print(f' Total no. of dataset that are sampled using the WeightedRandomSampler : { total_samples }')

 Total no. of dataset that are sampled using the WeightedRandomSampler : 50000


In [10]:
dl = DataLoader(train_ds, batch_size=32, num_workers = 2, sampler = weighted_sampler)
val_dl = DataLoader(val_ds, batch_size=32, num_workers = 2, drop_last=True)
print('Using DataLoaders to Load the training and validation dataset')

Using DataLoaders to Load the training and validation dataset


In [11]:
import torch.nn as nn

class CNN6(nn.Module):
    def __init__(self):
        super().__init__()

        # Convolution layer for input size (3X248X496)
        self.network = nn.Sequential(
            nn.Conv2d(3, 32, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2, 2),

            nn.Conv2d(64, 128, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(128),
            nn.ReLU(),
            nn.Dropout(0.2),
            nn.Conv2d(128, 128, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(128),
            nn.ReLU(),
            nn.Dropout(0.2),
            nn.MaxPool2d(2, 2),

            nn.Conv2d(128, 256, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(256),
            nn.ReLU(),
            nn.Dropout(0.2),
            nn.Conv2d(256, 256, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(256),
            nn.ReLU(),
            nn.Dropout(0.2),
            nn.MaxPool2d(2, 2),

            nn.Conv2d(256, 512, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(512),
            nn.ReLU(),
            nn.Dropout(0.2),
            nn.Conv2d(512, 512, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(512),
            nn.ReLU(),
            nn.Dropout(0.2),
            nn.MaxPool2d(2, 2),

            nn.Flatten(),
            nn.Linear(512 * 15 * 31, 1024),
            nn.BatchNorm1d(1024),
            nn.ReLU(),
            nn.Dropout(0.2),
            nn.Linear(1024, 512),
            nn.BatchNorm1d(512),
            nn.ReLU(),
            nn.Dropout(0.2),
            nn.Linear(512, 4),
            nn.Softmax(dim=1)
        )

    def forward(self,x):
        return self.network(x)
model = CNN6()
print('Initialized CNN 4_5 model')

Initialized CNN 4_5 model


In [1]:
model = DataParallel(model)
print('Model is wrapped with DataParallel object')

# Move the model to the GPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)
print('Sending the model to GPPU')

# Define the loss function
criterion = nn.CrossEntropyLoss()

# Define the optimizer
optimizer = optim.Adam(model.parameters())

# Define the number of epochs
num_epochs = 25
print(f'No. of epochs : { num_epochs }')

# dl, val_dl, model, optimizer = accelerator.prepare(dl, val_dl, model, optimizer)

# Training loop
for epoch in range(num_epochs):
    # Training phase
    model.train()
    train_loss = 0.0
    train_correct = 0
    train_total = 0

    for inputs, labels in dl:
        inputs = inputs.to(device)
        labels = labels.to(device)

        # Zero the gradients
        optimizer.zero_grad()

        # Forward pass
        outputs = model(inputs)
        loss = criterion(outputs, labels)

        # Backward pass and optimization
        loss.backward()
#         accelerator.backward(loss)
        optimizer.step()

        # Compute training accuracy
        _, predicted = torch.max(outputs.data, 1)
        train_total += labels.size(0)
        train_correct += (predicted == labels).sum().item()

        train_loss += loss.item()

    train_accuracy = 100 * train_correct / train_total
    print(f'Trainng of epoch {epoch} is completed !')

    # Validation phase
    model.eval()
    val_loss = 0.0
    val_correct = 0
    val_total = 0

    with torch.no_grad():
        for inputs, labels in val_dl:
            inputs = inputs.to(device)
            labels = labels.to(device)

            # Forward pass
            outputs = model(inputs)
            loss = criterion(outputs, labels)


            # Compute validation accuracy
            _, predicted = torch.max(outputs.data, 1)
            val_total += labels.size(0)
            val_correct += (predicted == labels).sum().item()

            val_loss += loss.item()

    val_accuracy = 100 * val_correct / val_total

    # Print epoch statistics
    print(f"Epoch [{epoch+1}/{num_epochs}]: "
          f"Train Loss: {train_loss:.4f}, Train Accuracy: {train_accuracy:.2f}%, "
          f"Val Loss: {val_loss:.4f}, Val Accuracy: {val_accuracy:.2f}%")
    if epoch > 10 :
        torch.save(model.state_dict(), f'model_{epoch+1}.pth')


NameError: ignored

In [ ]:
torch.save(model.state_dict(), 'finalized_model.pth')

In [ ]:
model.parameters

In [ ]:
torch.save(model.state_dict(), 'finalized_model.pth')

In [ ]:
import pickle
filename = 'finalized_model.sav'
pickle.dump(model, open(filename, 'wb'))